In [14]:
import math
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import *
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
df=pd.read_csv("Cryto_daily.csv")
print("Number of rows and columns:", df.shape)
df.head()

Number of rows and columns: (2949, 10)


,time,high,low,open,volumefrom,volumeto,close,conversionType,Datetime,Year
0,1388620800,886.21,810.47,815.94,12811.63,10826907.96,856.91,direct,2014-01-02,2014
1,1388707200,888.23,839.44,856.91,9709.01,8409629.11,884.26,direct,2014-01-03,2014
2,1388793600,932.16,848.32,884.26,14239.01,12784626.88,924.69,direct,2014-01-04,2014
3,1388880000,1029.86,911.36,924.69,21374.20,20789364.52,1014.74,direct,2014-01-05,2014
4,1388966400,1093.44,964.74,1014.74,23076.53,23622237.65,1012.65,direct,2014-01-06,2014


In [5]:
training_set = df[df['Year'] < 2021].iloc[:, 6:7].values
test_set = df[df['Year'] >= 2021].iloc[:, 6:7].values


In [6]:
# Feature Scaling
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

# Creating a data structure with 60 time-steps and 1 output
X_train = []
y_train = []

for i in range(60, 2556):

    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [7]:
# Getting the predicted stock price of 2017
dataset_train = df.iloc[:2556, 1:2]
dataset_test = df.iloc[2556:, 1:2]

dataset_total = pd.concat((dataset_train, dataset_test), axis = 0)
dummy = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values

inputs = dummy.reshape(-1,1)
inputs_transformed = sc.transform(inputs)

X_test = []

for i in range(60, 453):
    X_test.append(inputs_transformed[i-60:i, 0])
    
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
print(X_test.shape)

(393, 60, 1)


In [16]:
model_loaded = tf.keras.models.load_model('Model_Testing/Crypto_Models/Trained_model_2_daily_BTC_4L_50N_0p1D_trainUpTo2021.h5', compile = False)

In [17]:
predicted_stock_price = model_loaded.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [30]:
from flask import jsonify
import json

list_pred = list(predicted_stock_price[:,0])

pred_dict = {
    'predictions': [float(x) for x in list_pred]
}

with open('Crypto_2021_Predictions.json', 'w') as fp:
    json.dump(pred_dict, fp,  indent=4)
    


In [ ]:
# close_data = close_data.reshape((-1))

# def predict(num_prediction, model):
#     prediction_list = close_data[-look_back:]
    
#     for _ in range(num_prediction):
#         x = prediction_list[-look_back:]
#         x = x.reshape((1, look_back, 1))
#         out = model.predict(x)[0][0]
#         prediction_list = np.append(prediction_list, out)
#     prediction_list = prediction_list[look_back-1:]
        
#     return prediction_list
    
# def predict_dates(num_prediction):
#     last_date = df['Date'].values[-1]
#     prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
#     return prediction_dates

# num_prediction = 30
# forecast = predict(num_prediction, model)
# forecast_dates = predict_dates(num_prediction)
# view raw